In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import math 
import numpy as np
from torch import nn
from torch.nn import functional as F
from torch.nn import ModuleList, ParameterList, Parameter
from torch.autograd import Variable, Function
from torch.nn import Linear, Module, LSTM, Parameter, BatchNorm2d, Conv2d, ConvTranspose2d, Softmax, RNN, RNNBase, RNNCell, RNNCellBase
from torch import zeros,zeros_like, ones_like, cat, ByteTensor, FloatTensor, rand, log, sigmoid, ones
from torch import add,tanh,squeeze,Tensor,float,stack, argmax
from torch import max as torchmax
from torch import min as torchmin
from torch.autograd import Variable
from torch import cuda, no_grad, save, load, cat, zeros
from torch.optim import Adam
import torch 

class trainer(object):
	def __init__(self,p,model):
		self.p = p
		self.model = model
		self.optimizer = Adam(self.model.parameters(), lr=self.model.p['lr'], weight_decay=1e-5)

# thi is what you would build
class pc_conv_network(Module):
	def __init__(self,p):
		super(pc_conv_network, self).__init__()

		# # will want to increase receptive field size as ascend
		# for i in range(p['layers']):
		# 	pc1 = pc_conv_layer(p['batch_size'], p['chan'][2], p['chan'][1], p['imdim'][2], p['nf'][2], 1, 0.0005)
		# 	pc2 = pc_conv_layer(p['batch_size'], p['chan'][3], p['chan'][2], p['imdim'][3], p['nf'][3], 1, 0.0005)
		# who knows if need batch norm
		self.bs = p['bs']
		self.iter = p['iter']
		self.nlayers = p['layers']
		self.chan = p['chan']
		self.imdim = p['imdim']
		self.imchan = p['imchan']

		self.F = None
		self.F_last = None

		self.baseline = None

		self.conv_trans = ModuleList(
			[ConvTranspose2d(p['chan'][i+1], p['chan'][i], p['ks'][i], 1,p['pad'][i])
			for i in range(self.nlayers)])

		# phi 
		# phi = 0 uses -1 (image), 0 and 1
		# phi = 2 uses 1, 2 and 3
		phi = []
		for i in range(self.nlayers):
			phi.append(nn.Parameter(torch.rand(p['bs'],self.chan[i+1] * self.imdim[i] * self.imdim[i] )))
		#phi.append(nn.Parameter(torch.ones(p['bs'],self.chan[self.nlayers] * self.imdim[self.nlayers]^2)))
		self.phi = nn.ParameterList(phi)
		self.top_cause = torch.ones_like(phi[self.nlayers-1])

		# should be one Sigma matrix over whole batch ***
		self.Sigma = nn.ParameterList([nn.Parameter(torch.diag(torch.ones(self.chan[i+1] * self.imdim[i+1] * self.imdim[i+1])))
			 for i in range(-1,self.nlayers)])

		self.optimizer = Adam(self.parameters(), lr=p['lr'], weight_decay=1e-5)

	def loss(self, i):

		
		self.F = 0

		Theta__h_of_phi = (self.conv_trans[i](F.relu(self.phi[i].view(self.bs, self.chan[i+1], self.imdim[i+1], self.imdim[i+1])))).view(self.bs,-1)

		if i == self.nlayers-1:
			Theta__h_of_phi_above = self.top_cause

		else:
			Theta__h_of_phi_above = (self.conv_trans[i+1](F.relu(self.phi[i+1].view(self.bs, self.chan[i+2], self.imdim[i+2], self.imdim[i+2])))).view(self.bs,-1)
	 
		if i > 0:

			self.F += - torch.sum(0.5*(
				- torch.logdet(self.Sigma[i+1])
				- torch.squeeze(torch.matmul(torch.matmul(
					(self.phi[i] - Theta__h_of_phi_above).unsqueeze(1),
					torch.inverse(self.Sigma[i+1])),
				(self.phi[i] - Theta__h_of_phi_above).unsqueeze(2)))

				- torch.logdet(self.Sigma[i]) 
				- torch.squeeze(torch.matmul(torch.matmul(
					(self.phi[i-1] - Theta__h_of_phi).unsqueeze(1),   
					torch.inverse(self.Sigma[i])),	
				(self.phi[i-1] - Theta__h_of_phi).unsqueeze(2)))
				))
		if i == 0:
			self.F += - torch.sum(0.5*(	 # minus here so treated as loss
				- torch.logdet(self.Sigma[i+1])
				- torch.squeeze(torch.matmul(torch.matmul(
					(self.phi[i] - Theta__h_of_phi_above).unsqueeze(1), 
					torch.inverse(self.Sigma[i+1])),
				(self.phi[i] - Theta__h_of_phi_above).unsqueeze(2)))

				- torch.logdet(self.Sigma[i]) 
				- torch.squeeze(torch.matmul(
					torch.matmul((self.images - Theta__h_of_phi).unsqueeze(1),   
					torch.inverse(self.Sigma[i])),
				(self.images - Theta__h_of_phi).unsqueeze(2)))
				))
		if i == 0:
			self.estimate = Theta__h_of_phi

		

	def inference(self, i):

		self.conv_trans.requires_grad_(False)
		self.Sigma.requires_grad_(False)
		self.phi.requires_grad_(True)

		self.optimizer.zero_grad()
		self.loss(i)
		self.F.backward()
		self.optimizer.step()

	def learn(self,i):

		self.conv_trans.requires_grad_(True)
		self.Sigma.requires_grad_(True)
		self.phi.requires_grad_(False)

		self.optimizer.zero_grad()
		self.loss(i)
		self.F.backward()
		self.optimizer.step()


	def forward(self, images, learn=0):
		self.F_last = self.F
		self.images = images.view(self.bs, -1)
		for n in range(self.iter):
			print(n)
			if self.F is not None:
				self.F_last = self.F
			for i in range(self.nlayers):
				self.inference(i)

			
			print(self.F)
			print(torch.sum(self.images-self.estimate))
			if self.F_last is not None:
				print(self.F - self.F_last)
				if self.F > self.F_last:
					#self.optimizer = Adam(self.parameters(), lr=p['lr']/5, weight_decay=1e-5)
					break
		if learn:
			for i in range(self.nlayers):
				self.optimizer = Adam(self.parameters(), lr=0.001, weight_decay=1e-5)
				self.learn(i)


In [0]:
p = dict()
p['layers'] = 3
p['chan'] = dict()
p['imchan'] = 1
p['chan'][0] = 1
p['chan'][1] = 4
p['chan'][2] = 8
p['chan'][3] = 16

p['pad'] = dict()
p['pad'][0] = 1
p['pad'][1] = 1
p['pad'][2] = 1
p['pad'][3] = 1

p['ks'] = dict()
p['ks'][0] = 5
p['ks'][1] = 5
p['ks'][2] = 9

p['iter'] = 200
p['bs'] = 200
# image size - can prob be done automatically
p['imdim'] = dict()
p['imdim'][0] = 21
p['imdim'][1] = 21
p['imdim'][2] = 21
p['imdim'][3] = 21
p['pad'] = dict()
p['pad'][0] = 2
p['pad'][1] = 2
p['pad'][2] = 4

p['lr'] = 0.05
a = pc_conv_network(p)
#batch_size, number_of_kernels, w, h].


In [0]:
a = pc_conv_network(p)
pred = torch.rand(p['bs'],p['chan'][0]*p['imdim'][0],p['imdim'][0])
#p['lr'] = 0.04
F= a(pred, learn =1)


0
tensor(378844.5625, grad_fn=<AddBackward0>)
tensor(38188.3750, grad_fn=<SumBackward0>)
1
tensor(299320.3438, grad_fn=<AddBackward0>)
tensor(36080.9062, grad_fn=<SumBackward0>)
tensor(-79524.2188, grad_fn=<SubBackward0>)
2
tensor(253078.2188, grad_fn=<AddBackward0>)
tensor(35138.4258, grad_fn=<SumBackward0>)
tensor(-46242.1250, grad_fn=<SubBackward0>)
3
tensor(220760.5312, grad_fn=<AddBackward0>)
tensor(34450.3594, grad_fn=<SumBackward0>)
tensor(-32317.6875, grad_fn=<SubBackward0>)
4
tensor(192178.8750, grad_fn=<AddBackward0>)
tensor(33745.4688, grad_fn=<SumBackward0>)
tensor(-28581.6562, grad_fn=<SubBackward0>)
5
tensor(164317., grad_fn=<AddBackward0>)
tensor(32858., grad_fn=<SumBackward0>)
tensor(-27861.8750, grad_fn=<SubBackward0>)
6
tensor(137731.4219, grad_fn=<AddBackward0>)
tensor(31699.7500, grad_fn=<SumBackward0>)
tensor(-26585.5781, grad_fn=<SubBackward0>)
7
tensor(113702.5703, grad_fn=<AddBackward0>)
tensor(30280.6816, grad_fn=<SumBackward0>)
tensor(-24028.8516, grad_fn=<Sub

In [0]:
a.Sigma[2]

NameError: ignored

In [0]:
import math 
import numpy as np
from torch.nn import functional as F
from torch.nn import ModuleList
from torch.autograd import Variable, Function
from torch.nn import Linear, Module, LSTM, Parameter, BatchNorm2d, Conv2d, ConvTranspose2d, Softmax, RNN, RNNBase, RNNCell, RNNCellBase
from torch import zeros,zeros_like, ones_like, cat, ByteTensor, FloatTensor, rand, log, sigmoid
from torch import add,tanh,squeeze,Tensor,float,stack, argmax
from torch import max as torchmax
from torch import min as torchmin
from torch.autograd import Variable
from torch import cuda, no_grad, save, load, cat, zeros
from torch.optim import Adam
import torch 

class trainer(object):
	def __init__(self,p,model):
		self.p = p
		self.model = model
		self.optimizer = Adam(self.model.parameters(), lr=self.model.p['lr'], weight_decay=1e-5)


#	def train:



class pc_conv_layer(Module):
	def __init__(self,p,l):
		super(pc_conv_layer, self).__init__()
		#basics
		self.p = p
		self.l = l
		self.dim = p['ldim'][l]
		self.bs = p['batch_size']
		self.imdim = p['imdim'][l]
		self.dt = 1/p['inf_iter'] # integration step based on n interations

		# Inference Parameters
		self.phi = torch.nn.Parameter(torch.rand([self.bs, self.dim * self.imdim * self.imdim]),requires_grad=True) 

		# Learning parameters
		#self.conv = Conv2d(p['ldim'][l], p['ldim'][l+1], p['nf'][l], padding=1)
		self.conv_trans = ConvTranspose2d(p['ldim'][l], p['ldim'][l-1], p['nf'][l], padding=1)
		self.Sigma = torch.nn.Parameter(torch.diag(torch.ones(self.dim * self.imdim * self.imdim)).unsqueeze(0).repeat(self.bs,1,1),requires_grad=True)

		# optimizer
		self.optimizer = Adam(self.parameters(), lr=self.p['lr'], weight_decay=1e-5)


	def loss(self):

		# g(phi,Theta) = Theta*h(phi) = conv_trans(h(phi))
		# hi_above are already in correct form
		self.F = (torch.sum(0.5*(	
					- torch.logdet(self.Sigma)
					- torch.matmul(
						torch.matmul(
							(self.phi - self.phi_above).unsqueeze(1),
							torch.inverse(self.Sigma)
						),
						(self.phi - self.phi_above).unsqueeze(2)
					)
					- torch.logdet(self.Sigma_below) 
					- torch.squeeze(torch.matmul(
						torch.matmul(						
							(self.u - (self.conv_trans(self.phi.view(self.bs, self.dim, self.imdim, self.imdim))).view(self.bs,-1)).unsqueeze(1),#(torch.transpose(self.uu_gpt,1,2)),   
							torch.inverse(self.Sigma_below)				# works for batch!
						),
						(self.u - (self.conv_trans(self.phi.view(self.bs, self.dim, self.imdim, self.imdim))).view(self.bs,-1)).unsqueeze(2),#(torch.transpose(self.uu_gpt,1,2)),   
					))
				)))

	def inference(self):
		# reset activities prior to starting (or not)
		# certainly need to reset gradients

		# need grads for theta*h'(e); that said for now the weights don't need to be differentiated..
		# NB h(vi) depends only on vi

		self.conv_trans.weight.requires_grad = False 
		self.conv_trans.bias.requires_grad = False
		self.Sigma.requires_grad = False
		self.phi.requires_grad = True

		self.optimizer.zero_grad()
		self.loss()
		self.F.backward()
		self.optimizer.step()

		#self.total_loss += self.F.item()

		# what do i want to get out to pass to other layers
		# theta*h'(e)

	def forward(self, phi_above, u, Sigma_below):  
		self.phi_above = phi_above.view(self.bs,-1)
		self.Sigma_below = Sigma_below
		self.u = u
		self.inference()

		# outputs to send to adjacent layers
		# e and u have now been updated by Adam

		#return self.u, self.conv(e)

		# need to add in batch norm
		# need to be able to turn grads on and off
		# eventually will want to incorporate this into one step for all layers, then propogate updated values
		# so just want one iteration in inference?
		# so have training iteration outside this class?

In [0]:
p = dict()
p['ldim'] = dict()
p['ldim'][1] = 10
p['ldim'][2] = 5
p['nf'] = dict()
p['nf'][2] = 3
p['inf_iter'] = 20
p['batch_size'] = 5
p['imdim'] = dict()
p['imdim'][1] = 15
p['imdim'][2] = 15

p['lr'] = 0.0005
a = pc_conv_layer(p,2)
#batch_size, number_of_kernels, w, h].
phi_above = torch.rand(p['batch_size'],p['ldim'][2],p['imdim'][2],p['imdim'][2])
u = torch.rand(p['batch_size'],p['ldim'][1]*p['imdim'][1]*p['imdim'][1])
Sigma_below = torch.diag(torch.ones(p['ldim'][1] * p['imdim'][1] * p['imdim'][1])).unsqueeze(0).repeat(p['batch_size'],1,1)
a(phi_above,u,Sigma_below)


In [0]:
a.F

tensor(-12571.1055, grad_fn=<SumBackward0>)

In [0]:
% cd /content/drive/My\ Drive/vae_prednet/cocoapi
!pip install kornia
#!curl https://sdk.cloud.google.com | bash
#!git clone https://github.com/cocodataset/cocoapi.git

/content/drive/My Drive/vae_prednet/cocoapi
     |████████████████████████████████| 122kB 6.5MB/s 


In [0]:
!pwd
!gsutil -m rsync gs://images.cocodataset.org/train2017 train

/content/drive/My Drive/vae_prednet/cocoapi
Building synchronization state...
At source listing 10000...
At source listing 20000...
At source listing 30000...
At source listing 40000...
Caught non-retryable exception while listing file://train: [Errno 5] Input/output error: 'train'
At source listing 50000...
At source listing 60000...
At source listing 70000...
At source listing 80000...
At source listing 90000...
At source listing 100000...
At source listing 110000...
CommandException: Caught non-retryable exception - aborting rsync


/content/drive/My Drive/vae_prednet/cocoapi
/content/drive/My Drive/vae_prednet/cocoapi


In [0]:
!python main.py -m stl10_patch.yaml

2019-10-31:23:13:02,488 INFO     [main.py:25] Namespace(config='stl10_patch.yaml')
2019-10-31:23:13:02,491 INFO     [main.py:39] Training
2019-10-31:23:13:02,491 INFO     [main.py:40] Reading permutation : 0 of 1
{'b': 200,
 'beta': 10.0,
 'conv': True,
 'dataset': 'stl10',
 'dynamic': False,
 'e': 25000,
 'early_stopping': True,
 'elbo_loss': True,
 'enc_h': [100, 600],
 'enc_temp': 0.67,
 'err_noise': True,
 'exp_name': 'stl10_patch',
 'foveate': False,
 'generate_data': False,
 'gpu': True,
 'h_label': '100',
 'imdim': (1, 7, 7),
 'interactive': False,
 'layers': 1,
 'lr': 0.0005,
 'model_name': 'l1_stl10_pnt0_vae1_normal_z75_h100_c100000_b10_ld100_elboTrue',
 'n_actions': 2,
 'nz_con': [65, 0, 0],
 'nz_dis': [[10], 0, 0],
 'patience': 5,
 'plot_iter': 500,
 'prednet': False,
 'rotating': False,
 'sb_patch_size': 7,
 'sequences': False,
 'train_fx': False,
 'train_gx': True,
 'use_lstm': False,
 'vae': True,
 'x_dist': 'normal',
 'z_con_capacity': [[0.0, 10.0, 100000, 100.0]],
 'z_d